In [ ]:
import requests
import json
import os
from requests.auth import HTTPBasicAuth
from tcpSDK import client
import tcpSDK

### Host Definition

In [ ]:
host="https://unstable.thecrossproduct.xyz/v1"

### Connection information

In [ ]:
email=""
password = ""

### Authentification

In [ ]:
api=client(host=host,usermail=email,passwd=password)

### Token recovery

In [ ]:
token=api.token
print(token)
#you can add the token to the env variable "TCP_API_TOKEN". 
#After that, you won't need to provide email and password to create a connection

### Get Users Information

In [ ]:
user_info=api.query().auth.get()
print(user_info)

## Token Information

In [ ]:
print(api.query().auth.validate_auth_token.get())


## List TCP Solutions

In [ ]:
print(api.query().app.get())

## Specific informations for Domain

In [ ]:
print(api.query().app.info.get(Domain="railway",App="classification").decode('utf8'))

### List Available remotes

In [ ]:
print(api.query().app.remote.scw.get())

## Upload Data

In [ ]:
#Définition des paramètres
uri_cible="test_upload"
file_path="/mnt/HDD/tcp_data/0_tests/test_cloud1/01_Sources/cloud1.laz"
file_size=str(os.path.getsize(file_path))

In [ ]:
api.upload(file_path,uri_cible)

## TCP List Data

In [ ]:
print(api.query().data.get())

## Run TCP Solution

In [ ]:
#définition des inputs
clouds=["demo/cloud1.laz"]
trj=["demo/trajectory.dxf"]
output="demo/classif"

In [ ]:

print(api.query().app.run.post({"inputs":{"pcs":clouds,"traj":trj},"output-prefix": output, "pool":["scw:fr-par-1:PRO2-M"]},Domain="railway",App="classification"))

In [ ]:
url = host+"v1/app/run?Domain=Railway&App=Classification"
payload = json.dumps({"inputs":{"pcs":clouds,"traj":trj},"output-prefix": output, "pool":["scw:fr-par-1:PRO2-M"] })
headers = {'Content-Type': 'application/json','x-access-token': token}

mod = requests.request("POST", url, headers=headers, data=payload)

if mod.status_code==200:
    print(mod.text)
else:
    print(mod)

## Check Process status & Download results

In [ ]:
id_process = mod.json()['id']

url = host+"/v1/app/status?Process={}".format(id_process)

headers = {
  'x-access-token': token
}

download_result = requests.request("GET", url, headers=headers)

print(download_result.text)
#le paramètre "output" contiendra un url vers le résultat à télécharger lorsque le traitement sera terminé

## List Processes

In [ ]:
print(api.query().app.list.Process.get())